### FINAL PROJECT: NYPD Arrests Data (Historic) 
# Dataset ETL

In [1]:
# Import dependencies
import pandas as pd
from google.colab import files

In [2]:
#  Read in the dataset into a DataFrame

# Sample (1000 records) of the dataset
#arrest_raw_df = pd.read_csv("https://data.cityofnewyork.us/resource/8h9b-rp9u.csv")

# Full dataset (~4 min.)
arrest_raw_df = pd.read_csv("https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD")

print(arrest_raw_df.shape)
arrest_raw_df.head()

(5308876, 19)


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,236791704,11/22/2021,581.0,NaN,NaN,NaN,PL 2225001,M,M,28,0.0,45-64,M,BLACK,997427.0,230378.0,40.799009,-73.952409,POINT (-73.95240854099995 40.799008797000056)
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,0.0,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,0.0,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
3,32311380,06/18/2007,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,27,1.0,18-24,M,BLACK,NaN,NaN,NaN,NaN,NaN
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)


#### Initial Dataset Analysis

In [3]:
# Examine the data types
arrest_raw_df.dtypes

ARREST_KEY             int64
ARREST_DATE           object
PD_CD                float64
PD_DESC               object
KY_CD                float64
OFNS_DESC             object
LAW_CODE              object
LAW_CAT_CD            object
ARREST_BORO           object
ARREST_PRECINCT        int64
JURISDICTION_CODE    float64
AGE_GROUP             object
PERP_SEX              object
PERP_RACE             object
X_COORD_CD           float64
Y_COORD_CD           float64
Latitude             float64
Longitude            float64
Lon_Lat               object
dtype: object

In [4]:
# Examine # of unique values for each column(feature)
arrest_raw_df.nunique()

ARREST_KEY           5308876
ARREST_DATE             5844
PD_CD                    340
PD_DESC                  424
KY_CD                     75
OFNS_DESC                 88
LAW_CODE                2481
LAW_CAT_CD                 4
ARREST_BORO                5
ARREST_PRECINCT           78
JURISDICTION_CODE         27
AGE_GROUP                 91
PERP_SEX                   2
PERP_RACE                  8
X_COORD_CD             65034
Y_COORD_CD             68029
Latitude              108390
Longitude             109604
Lon_Lat               124549
dtype: int64

In [5]:
# Display all unique values for "JURISDICTION_CODE"
print(arrest_raw_df['JURISDICTION_CODE'].unique())


[ 0.  1. 72.  2.  4. 73.  7. 71. 15. 97. 14.  3. 11.  6. 88. 87. 16. 13.
  9. 12. 79. 85. 69. 74. 76.  8. nan 82.]


In [6]:
# Display all unique values for "AGE_GROUP"
print(arrest_raw_df['AGE_GROUP'].unique())
print("\n")
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(arrest_raw_df.AGE_GROUP.value_counts())

['45-64' '25-44' '18-24' '<18' '65+' nan '929' '944' '945' '320' '932'
 '894' '935' '934' '940' '339' '323' '965' '948' '946' 'UNKNOWN' '938'
 '895' '194' '928' '352' '943' '956' '920' '951' '708' '816' '316' '912'
 '237' '939' '947' '740' '952' '949' '338' '922' '933' '921' '1042' '2000'
 '1937' '1916' '1928' '2007' '1918' '1947' '1998' '1953' '1942' '1024'
 '1948' '937' '1922' '1048' '1018' '918' '927' '926' '931' '942' '924'
 '941' '930' '955' '936' '808' '812' '317' '909' '923' '914' '231' '896'
 '709' '640' '925' '212' '200' '954' '314' '723' '309' '959' '910' '446'
 '330']


25-44      2482601
18-24      1375134
45-64       981931
<18         422931
65+          46084
895             13
945              7
894              7
935              7
UNKNOWN          5
937              5
928              5
931              4
924              4
922              4
943              4
927              4
929              4
932              4
948              3
936              3
896          

In [7]:
# Display all unique values for "PERP_SEX"
print(arrest_raw_df['PERP_SEX'].unique())
print("\n")
print(arrest_raw_df.PERP_SEX.value_counts())

['M' 'F']


M    4417407
F     891469
Name: PERP_SEX, dtype: int64


In [8]:
# Display all unique values for "PERP_RACE"
print(arrest_raw_df['PERP_RACE'].unique())
print("\n")
print(arrest_raw_df.PERP_RACE.value_counts())

['BLACK' 'WHITE HISPANIC' 'UNKNOWN' 'WHITE' 'BLACK HISPANIC'
 'ASIAN / PACIFIC ISLANDER' 'AMERICAN INDIAN/ALASKAN NATIVE' 'OTHER']


BLACK                             2579848
WHITE HISPANIC                    1376387
WHITE                              637703
BLACK HISPANIC                     428839
ASIAN / PACIFIC ISLANDER           221280
UNKNOWN                             51587
AMERICAN INDIAN/ALASKAN NATIVE      11869
OTHER                                1363
Name: PERP_RACE, dtype: int64


#### Delete Unnecessary Columns

In [9]:
# Drop the "JURISDICTION_CODE"
arrest_good_columns_df = arrest_raw_df.drop(["JURISDICTION_CODE"], axis=1)

# Verify in DF
print(arrest_good_columns_df.shape)
arrest_good_columns_df.head()

(5308876, 18)


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,236791704,11/22/2021,581.0,NaN,NaN,NaN,PL 2225001,M,M,28,45-64,M,BLACK,997427.0,230378.0,40.799009,-73.952409,POINT (-73.95240854099995 40.799008797000056)
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
3,32311380,06/18/2007,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,27,18-24,M,BLACK,NaN,NaN,NaN,NaN,NaN
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)


#### Handle Invalid Age Group Assigned

In [10]:
# Establish Valid Age Groups
valid_age_groups = ["25-44", "18-24", "45-64", "<18", "65+"]

# Delete Rows with Invalid Age Groups
arrest_good_age_df = arrest_good_columns_df[~arrest_raw_df["AGE_GROUP"].isin(valid_age_groups) == False]

print(arrest_good_age_df.shape)
print("\n")
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(arrest_good_age_df.AGE_GROUP.value_counts())

(5308681, 18)


25-44    2482601
18-24    1375134
45-64     981931
<18       422931
65+        46084
Name: AGE_GROUP, dtype: int64


#### Handle Null Data in Dataset

In [11]:
# Check for null values
arrest_good_age_df.isnull().sum()

ARREST_KEY             0
ARREST_DATE            0
PD_CD                309
PD_DESC             9165
KY_CD               9165
OFNS_DESC           9165
LAW_CODE             192
LAW_CAT_CD         20250
ARREST_BORO            8
ARREST_PRECINCT        0
AGE_GROUP              0
PERP_SEX               0
PERP_RACE              0
X_COORD_CD             1
Y_COORD_CD             1
Latitude               1
Longitude              1
Lon_Lat                1
dtype: int64

In [12]:
# Drop all rows with any null data
arrest_no_null_df = arrest_good_age_df.dropna()

print(arrest_no_null_df.shape)
arrest_no_null_df.head()

(5279449, 18)


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
7,236106641,11/10/2021,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,B,41,25-44,M,WHITE HISPANIC,1017934.0,232221.0,40.804013,-73.878332,POINT (-73.87833183299993 40.804012949000025)
8,238383628,12/28/2021,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,Q,113,18-24,M,BLACK,1045482.0,191341.0,40.691660,-73.779199,POINT (-73.77919852099996 40.69166001700007)


In [13]:
# Verify null row removal
arrest_no_null_df.isnull().sum()

ARREST_KEY         0
ARREST_DATE        0
PD_CD              0
PD_DESC            0
KY_CD              0
OFNS_DESC          0
LAW_CODE           0
LAW_CAT_CD         0
ARREST_BORO        0
ARREST_PRECINCT    0
AGE_GROUP          0
PERP_SEX           0
PERP_RACE          0
X_COORD_CD         0
Y_COORD_CD         0
Latitude           0
Longitude          0
Lon_Lat            0
dtype: int64

In [14]:
# Examine # of unique values for each column(feature) after NULL removal
arrest_no_null_df.nunique()

ARREST_KEY         5279449
ARREST_DATE           5844
PD_CD                  312
PD_DESC                405
KY_CD                   75
OFNS_DESC               87
LAW_CODE              2410
LAW_CAT_CD               4
ARREST_BORO              5
ARREST_PRECINCT         77
AGE_GROUP                5
PERP_SEX                 2
PERP_RACE                8
X_COORD_CD           64999
Y_COORD_CD           67995
Latitude            108317
Longitude           109525
Lon_Lat             124431
dtype: int64

#### Convert arrest date string to a date format

In [15]:
# Convert string to data type
arrest_no_null_df["ARREST_DATE"] = pd.to_datetime(arrest_no_null_df["ARREST_DATE"])

# Verify in data types
print(arrest_no_null_df.dtypes)

# Verify in DF
arrest_no_null_df

ARREST_KEY                  int64
ARREST_DATE        datetime64[ns]
PD_CD                     float64
PD_DESC                    object
KY_CD                     float64
OFNS_DESC                  object
LAW_CODE                   object
LAW_CAT_CD                 object
ARREST_BORO                object
ARREST_PRECINCT             int64
AGE_GROUP                  object
PERP_SEX                   object
PERP_RACE                  object
X_COORD_CD                float64
Y_COORD_CD                float64
Latitude                  float64
Longitude                 float64
Lon_Lat                    object
dtype: object


<ipython-input-15-b6228baa811e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrest_no_null_df["ARREST_DATE"] = pd.to_datetime(arrest_no_null_df["ARREST_DATE"])


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
1,237354740,2021-12-04,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,2021-11-09,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
4,192799737,2019-01-26,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
7,236106641,2021-11-10,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,B,41,25-44,M,WHITE HISPANIC,1017934.0,232221.0,40.804013,-73.878332,POINT (-73.87833183299993 40.804012949000025)
8,238383628,2021-12-28,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,Q,113,18-24,M,BLACK,1045482.0,191341.0,40.691660,-73.779199,POINT (-73.77919852099996 40.69166001700007)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5308871,24092381,2006-08-31,223.0,"BURGLARY,RESIDENCE,NIGHT",107.0,BURGLARY,PL 1402502,F,Q,115,25-44,M,WHITE HISPANIC,1012648.0,214098.0,40.754289,-73.897502,POINT (-73.89750191399997 40.75428869800004)
5308872,24157849,2006-09-05,397.0,"ROBBERY,UNCLASSIFIED,OPEN AREAS",105.0,ROBBERY,PL 1601001,F,B,40,18-24,M,BLACK,1009374.0,235406.0,40.812783,-73.909239,POINT (-73.90923937299993 40.81278337600002)
5308873,24419289,2006-09-21,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,M,17,25-44,F,WHITE,992411.0,215025.0,40.756875,-73.970544,POINT (-73.97054405699998 40.75687491100007)
5308874,24130128,2006-09-02,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,K,79,45-64,F,BLACK,1001326.0,186947.0,40.679795,-73.938436,POINT (-73.93843631199996 40.679794835000045)


#### Pull Out Database DataFrames and Export



In [16]:
# Get unique PD_CD/PD_DESC pair values and sort
pd_code_df = arrest_no_null_df.loc[:, ["PD_CD", "PD_DESC"]].drop_duplicates().sort_values(by=['PD_CD'])

# Drop all duplicate rows based on KY_CD, keeping the first description only
pd_code_df.drop_duplicates(subset=["PD_CD"], inplace=True)

# Reset Dataframe index
pd_code_df.reset_index(drop=True)

print(pd_code_df.shape)
pd_code_df   #.head()

(312, 2)


,PD_CD,PD_DESC
777948,11.0,"NY CITY,TRAFFIC SUMMONS WARRANT"
403842,100.0,STALKING COMMIT SEX OFFENSE
10,101.0,ASSAULT 3
754,104.0,VEHICULAR ASSAULT (INTOX DRIVE
12,105.0,STRANGULATION 1ST
...,...,...
790,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO"
632600,970.0,SPILLBACK
110778,972.0,SEAT BELTS
568799,973.0,USE OF CELLULAR TELEPHONE WHILE DRIVING


In [17]:
# Get unique KY_CD/OFNS_DESC pair values and sort
key_code_df = arrest_no_null_df.loc[:, ["KY_CD", "OFNS_DESC"]].drop_duplicates().sort_values(by=['KY_CD'])  #.values

# Drop all duplicate rows based on KY_CD, keeping the first description only
key_code_df.drop_duplicates(subset=["KY_CD"], inplace=True)

# Reset Dataframe index
key_code_df.reset_index(drop=True)

print(key_code_df.shape)
key_code_df   #.head()

(75, 2)


,KY_CD,OFNS_DESC
81743,101.0,MURDER & NON-NEGL. MANSLAUGHTER
15698,102.0,HOMICIDE-NEGLIGENT-VEHICLE
762,103.0,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
1,104.0,RAPE
17,105.0,ROBBERY
...,...,...
5820,685.0,ADMINISTRATIVE CODES
7118,880.0,MOVING INFRACTIONS
625,881.0,OTHER TRAFFIC INFRACTION
91348,882.0,PARKING OFFENSES


In [18]:
# Drop the description columns for a clean arrest dataset table
full_arrest_clean_df = arrest_no_null_df.drop(["PD_DESC", "OFNS_DESC"], axis=1).sort_values(by=['ARREST_DATE'])

print(full_arrest_clean_df.shape)
full_arrest_clean_df   #.head()

(5279449, 16)


,ARREST_KEY,ARREST_DATE,PD_CD,KY_CD,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
940173,9932920,2006-01-01,905.0,347.0,VTL11920M2,M,K,72,45-64,M,WHITE,982788.0,174192.0,40.644802,-74.005270,POINT (-74.00526971499994 40.64480152000005)
951174,9930289,2006-01-01,779.0,126.0,PL 215510D,F,Q,101,25-44,M,BLACK,1053650.0,158969.0,40.602746,-73.750078,POINT (-73.75007786499998 40.60274595100003)
952040,9930212,2006-01-01,567.0,235.0,PL 2211001,M,Q,114,25-44,M,WHITE HISPANIC,1005827.0,214404.0,40.755148,-73.922121,POINT (-73.92212058399997 40.75514786500003)
943429,9930043,2006-01-01,567.0,235.0,PL 2211001,M,K,83,18-24,M,BLACK,1004778.0,192079.0,40.693874,-73.925975,POINT (-73.92597497799993 40.69387366500007)
940891,9930038,2006-01-01,109.0,106.0,PL 1200501,F,K,61,25-44,M,WHITE,999271.0,152992.0,40.586599,-73.945921,POINT (-73.94592089799994 40.586599294000045)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374775,238479959,2021-12-31,101.0,344.0,PL 1200001,M,K,66,25-44,M,BLACK,986771.0,168542.0,40.629293,-73.990919,POINT (-73.99091920699995 40.62929323400005)
376025,238471407,2021-12-31,397.0,105.0,PL 1601502,F,K,72,18-24,M,WHITE HISPANIC,981169.0,177800.0,40.654704,-74.011105,POINT (-74.01110537999993 40.65470428400005)
374696,238479962,2021-12-31,439.0,109.0,PL 1553001,F,M,18,25-44,M,BLACK,988211.0,218129.0,40.765397,-73.985702,POINT (-73.98570226399994 40.765397475000036)
372172,238489820,2021-12-31,139.0,101.0,PL 1252501,F,M,32,18-24,M,BLACK,999428.0,236544.0,40.815930,-73.945167,POINT (-73.94516735699995 40.815929553000046)


In [19]:
# Shrink the dataset to arrest made in the last 10 years
arrest_10y_clean_df = full_arrest_clean_df.loc[(full_arrest_clean_df['ARREST_DATE'] >= '2012-01-01')]

print(arrest_10y_clean_df.shape)
arrest_10y_clean_df   #.head()

(2855141, 16)


,ARREST_KEY,ARREST_DATE,PD_CD,KY_CD,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
3132614,82422385,2012-01-01,759.0,359.0,PL 1950500,M,K,79,18-24,M,BLACK,997366.0,192877.0,40.696078,-73.952702,POINT (-73.95270199199997 40.69607807800003)
3131463,82422694,2012-01-01,101.0,344.0,PL 1200001,M,M,6,18-24,M,WHITE,983794.0,204737.0,40.728641,-74.001647,POINT (-74.00164669699996 40.72864066600005)
3118457,82422331,2012-01-01,905.0,347.0,VTL11920U2,M,K,72,25-44,M,WHITE HISPANIC,989519.0,178298.0,40.656070,-73.981012,POINT (-73.981011557 40.656070154000076)
3118535,82422817,2012-01-01,782.0,236.0,PL 2650101,M,Q,115,25-44,M,BLACK HISPANIC,1017897.0,219850.0,40.770058,-73.878527,POINT (-73.87852744499997 40.77005803100008)
3139043,82422890,2012-01-01,258.0,351.0,PL 1456002,M,Q,104,18-24,M,WHITE,1012109.0,195368.0,40.702881,-73.899525,POINT (-73.89952491499997 40.70288116000006)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374775,238479959,2021-12-31,101.0,344.0,PL 1200001,M,K,66,25-44,M,BLACK,986771.0,168542.0,40.629293,-73.990919,POINT (-73.99091920699995 40.62929323400005)
376025,238471407,2021-12-31,397.0,105.0,PL 1601502,F,K,72,18-24,M,WHITE HISPANIC,981169.0,177800.0,40.654704,-74.011105,POINT (-74.01110537999993 40.65470428400005)
374696,238479962,2021-12-31,439.0,109.0,PL 1553001,F,M,18,25-44,M,BLACK,988211.0,218129.0,40.765397,-73.985702,POINT (-73.98570226399994 40.765397475000036)
372172,238489820,2021-12-31,139.0,101.0,PL 1252501,F,M,32,18-24,M,BLACK,999428.0,236544.0,40.815930,-73.945167,POINT (-73.94516735699995 40.815929553000046)


In [ ]:
# Export all 3 datasets
pd_code_df.to_csv('pd_code_clean.csv', index=False)
files.download('pd_code_clean.csv')

key_code_df.to_csv('key_code_clean.csv', index=False)
files.download('key_code_clean.csv')

arrest_10y_clean_df.to_csv('arrest_10yr_clean.csv', index=False)
#files.download('arrest_10yr_clean.csv')
! zip arrest_10yr_clean.zip arrest_10yr_clean.csv  # Compress file and Export


#### Create a Sample of Clean Data for Analysis and Export

In [21]:
# Create a sample of 10,000 rows
arrest_sample_clean_df = arrest_10y_clean_df.sample(n=10000)

# Sort arrests by data
arrest_sample_clean_df = arrest_sample_clean_df.sort_values(by=['ARREST_DATE'])

print(arrest_sample_clean_df.shape)
arrest_sample_clean_df   #.head()

(10000, 16)


,ARREST_KEY,ARREST_DATE,PD_CD,KY_CD,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
3136936,82422660,2012-01-01,115.0,355.0,PL 1202000,M,M,26,18-24,M,BLACK,996463.0,235457.0,40.812951,-73.955881,POINT (-73.95588109399995 40.812950638000075)
3120902,82437191,2012-01-02,759.0,359.0,PL 1950500,M,B,52,18-24,M,WHITE HISPANIC,1011577.0,255024.0,40.866623,-73.901201,POINT (-73.90120096599996 40.86662251100006)
3142456,82450817,2012-01-02,567.0,235.0,PL 2211002,M,K,75,18-24,M,BLACK,1019569.0,185331.0,40.675305,-73.872672,POINT (-73.87267226099993 40.67530529600003)
3121184,82437288,2012-01-02,101.0,344.0,PL 1200001,M,K,71,25-44,M,BLACK,1001421.0,178336.0,40.656159,-73.938116,POINT (-73.93811573399995 40.65615937600007)
3127243,82450875,2012-01-03,113.0,344.0,PL 1201401,M,K,94,<18,M,BLACK,998469.0,201680.0,40.720239,-73.948706,POINT (-73.94870571999998 40.72023852700005)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356593,238423038,2021-12-29,793.0,118.0,PL 2650201,F,Q,106,45-64,M,ASIAN / PACIFIC ISLANDER,1033013.0,189339.0,40.686242,-73.824176,POINT (-73.82417615599996 40.68624248100008)
361110,238418800,2021-12-29,922.0,348.0,VTL0511001,M,M,26,25-44,M,WHITE HISPANIC,996463.0,235457.0,40.812951,-73.955881,POINT (-73.95588109399995 40.812950638000075)
377685,238478556,2021-12-31,793.0,118.0,PL 2650201,F,Q,109,25-44,M,BLACK,1030272.0,215099.0,40.756962,-73.833883,POINT (-73.83388324699997 40.75696201900007)
363607,238469348,2021-12-31,101.0,344.0,PL 1200001,M,B,47,25-44,M,BLACK,1026648.0,256536.0,40.870713,-73.846702,POINT (-73.84670244599994 40.870712966000035)


In [22]:
# Export DF
arrest_sample_clean_df.to_csv('arrest_10Ksample_clean.csv', index=False)
files.download('arrest_10Ksample_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>